In [1]:
import os
import pickle
from bs4 import BeautifulSoup
JAVA_SO_POSTS_STORE_PATH = 'C:/workspace/SOworkspace/data/so_posts/java/'
post_file_names = os.listdir(JAVA_SO_POSTS_STORE_PATH)
post_file_names[0:1]

['posts_0.pkl']

In [2]:
'''
从已经提取的句子里找点case做一下处理尝试
下面是选取的典型句子sample
'''
sample_text = '''<p>It's called the <a href=\"https://docs.oracle.com/javase/8/docs/technotes/guides/language/type-inference-generic-instance-creation.html\" rel=\"nofollow\">diamond operator</a>. It was introduced in Java 1.7.</p>&#xA;&#xA;<p>The benefit is just that you need to write less code.</p>&#xA;&#xA;<p>Compare</p>&#xA;&#xA;<pre><code>List&lt;Map&lt;String, Integer&gt;&gt; list = new ArrayList&lt;Map&lt;String, Integer&gt;&gt;();&#xA;</code></pre>&#xA;&#xA;<p>to</p>&#xA;&#xA;<pre><code>List&lt;Map&lt;String, Integer&gt;&gt; list = new ArrayList&lt;&gt;();&#xA;</code></pre>&#xA;'''
    

In [3]:
soup = BeautifulSoup(sample_text, 'lxml')
soup.text

"It's called the diamond operator. It was introduced in Java 1.7.\nThe benefit is just that you need to write less code.\nCompare\nList<Map<String, Integer>> list = new ArrayList<Map<String, Integer>>();\n\nto\nList<Map<String, Integer>> list = new ArrayList<>();\n\n"

In [4]:
for pre in soup.find_all('pre'):
    pre.extract()
soup.text

"It's called the diamond operator. It was introduced in Java 1.7.\nThe benefit is just that you need to write less code.\nCompare\n\nto\n\n"

In [5]:
soup

<html><body><p>It's called the <a href="https://docs.oracle.com/javase/8/docs/technotes/guides/language/type-inference-generic-instance-creation.html" rel="nofollow">diamond operator</a>. It was introduced in Java 1.7.</p>
<p>The benefit is just that you need to write less code.</p>
<p>Compare</p>

<p>to</p>

</body></html>

In [6]:
a_texts = [a.text for a in soup.find_all('a') if len(a.text) > 1]
a_texts

['diamond operator']

In [7]:
lines = soup.text.split('\n')
lines

["It's called the diamond operator. It was introduced in Java 1.7.",
 'The benefit is just that you need to write less code.',
 'Compare',
 '',
 'to',
 '',
 '']

In [8]:
candidate_lines = [line for line in lines if any([t for t in a_texts if t in line])]
candidate_lines

["It's called the diamond operator. It was introduced in Java 1.7."]

In [10]:
def get_all_indexes(sub : str, s : str):
    ret = []
    start_from_index = 0
    while start_from_index < len(s):
        sub_begin_index = s.find(sub,start_from_index)
        if sub_begin_index == -1:
            break
        sub_end_index = sub_begin_index + len(sub) - 1
        ret.append([sub_begin_index, sub_end_index])
        start_from_index = sub_end_index + 1
    return ret

get_all_indexes('diamond operator' , '''It's called the diamond operator. It was introduced in Java 1.7.''')

[[16, 31]]

In [11]:
trainset = []
for candidate in candidate_lines:
    train_data = {
        "text" : candidate,
        "label" : {
            "api" : {}
        }
    }
    labels = {}
    for a_text in a_texts:
        if a_text in candidate:
            labels[a_text] = get_all_indexes(a_text, candidate)
    train_data["label"]["api"] = labels
    trainset.append(train_data)
trainset

[{'text': "It's called the diamond operator. It was introduced in Java 1.7.",
  'label': {'api': {'diamond operator': [[16, 31]]}}}]

In [12]:
import json
json_str = json.dumps(trainset[0], ensure_ascii=False)

In [13]:
json_str
"""
训练集生成尝试完毕，上述代码将被总结并加入EUREKA-2
"""

'{"text": "It\'s called the diamond operator. It was introduced in Java 1.7.", "label": {"api": {"diamond operator": [[16, 31]]}}}'